<a href="https://colab.research.google.com/github/LeonMilosevic/home_credit/blob/main/home_credit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Home Credit Default Risk Competition

### Intro

Introduction: \

https://www.kaggle.com/c/home-credit-default-risk/overview

In Home Credit Default Risk Competition we need to make a model, that will give us a probability that the given client repays the loan. This should help the business decide if they should or should not proivde financial loan to a given customer.

Problem:

It is a binary classification problem:
- Will repay the loan = 0
- Will not repay the loan = 1

Evaluation Criteria:

Based on the requirements of the competition, we will need to evaluate our models on ***roc_auc_score*** metric.

Data:

- application_{train|test}.csv (main file)
- bureau.csv
- bureau_balance.csv
- POS_CASH_balance.csv
- credit_card_balance.csv
- previous_application.csv
- installments_payments.csv
- HomeCredit_columns_description.csv

Approach:
- Explore the data of application_train
- Create baseline model
- Explore other files
- Add usefull features from other files
- Test models on new features
- Feature Engineering


Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sn


Files

Custom Functions

### EDA